In [1]:
!pip -q install langchain langchain_community langchain-groq tiktoken chromadb huggingface_hub transformers sentence-transformers einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip show langchain

Name: langchain
Version: 0.1.17
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Texts
- Embedding
- ChromaDB
- QA Chain
- Source info

## Setting up LangChain


In [3]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_groq import ChatGroq
from glob import glob
import os


## Models to use

In [4]:
embedding_name = 'nomic-ai/nomic-embed-text-v1.5'

In [5]:
# Load embedding
model_kwargs = {'trust_remote_code': True}
embedding = HuggingFaceEmbeddings(model_name=embedding_name, model_kwargs=model_kwargs)

modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [6]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initializing GROQ chat with provided API key, model name, and settings
model = ChatGroq(
            groq_api_key=groq_api_key, model_name="llama3-70b-8192",
                         temperature=0.2)

## Load multiple and process documents

In [15]:
# Initialize documents and path
documents = []
root  = os.getcwd()

# Sort the file names
paths = sorted(glob('*.txt'))

# Copy each file to new file
for path in paths:
  loader = TextLoader(path)
  document = loader.load()
  documents.extend(document)

In [17]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [18]:
len(texts)

159

In [22]:
for text in texts:
  print(text.metadata)

{'source': '01-Description.txt'}
{'source': '01-Description.txt'}
{'source': '01-Description.txt'}
{'source': '02-Catholic Identity, Vision Mission and Core Values.txt'}
{'source': '02-Catholic Identity, Vision Mission and Core Values.txt'}
{'source': '02-Catholic Identity, Vision Mission and Core Values.txt'}
{'source': '03-The University Seal.txt'}
{'source': '04-History.txt'}
{'source': '04-History.txt'}
{'source': '04-History.txt'}
{'source': '04-History.txt'}
{'source': '05-University Saints.txt'}
{'source': '05-University Saints.txt'}
{'source': '05-University Saints.txt'}
{'source': '05-University Saints.txt'}
{'source': '06-USC Organizational Structure.txt'}
{'source': '06-USC Organizational Structure.txt'}
{'source': '06-USC Organizational Structure.txt'}
{'source': '07-Admission.txt'}
{'source': '07-Admission.txt'}
{'source': '08-Admission Steps for Freshmen.txt'}
{'source': '08-Admission Steps for Freshmen.txt'}
{'source': '09-Admission Steps for International Students.txt'}

## Create the DB

In [23]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [24]:
# persiste the db to disk
vectordb.persist()
vectordb = None

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [25]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## Make a retriever

In [26]:
retriever = vectordb.as_retriever()

In [27]:
docs = retriever.get_relevant_documents("What are the saints of University of San Carlos?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [28]:
len(docs)

4

In [29]:
for doc in docs:
  print(doc.metadata)

{'source': '04-History.txt'}
{'source': '05-University Saints.txt'}
{'source': '04-History.txt'}
{'source': '60-Directory of Student Support Services and Programs.txt'}


## Make a chain

In [30]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=model,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [37]:
## Cite sources
def process_llm_response(llm_response):
    if('According to the provided context, ' in llm_response['result']):
      llm_response['result'] = llm_response['result'][35:]
      llm_response['result'] = llm_response['result'][0].upper() + llm_response['result'][1:]
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [38]:
# full example
query = "What is the passing grade?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)

A passing grade is a grade within 1.0 and 3.0.


Sources:
37-Academic Performance and Grading System.txt
37-Academic Performance and Grading System.txt
39-Graduation.txt
35-Curricular and Non-Curricular Requirements.txt


In [39]:
# another example
query = "What is block enrollment?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)


Block enrollment is a type of enrollment that allows a student to enroll all the courses for the term at once. To be eligible for block enrollment, a student must be enrolling for a regular full load in a given semester, must not have incurred any failures or NG from previous semesters, and must have obtained passing grades for all courses that are prerequisites to the set of courses on a given semester.


Sources:
17-Enrollment Timeline.txt
17-Enrollment Timeline.txt
18-Enrollment Steps for Continuing Students.txt
18-Enrollment Steps for Continuing Students.txt


In [41]:
# another example
query = "How do I shift to another program?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)


To shift to another program, you need to follow these steps:

1. Click "Student Task", "Others", and "Apply Change of Program".
2. Select the option that applies to you between "Change Program (Same Department)" or "Change Program (Different Department)".
3. Indicate the academic period, year, school, department, and program where you want to shift to.
4. Place a reason for shifting to that program and then click "Submit".
5. Monitor the status at "Application History" which includes the old chair for endorsement, Counseling and Development Center for interview, the new chair for endorsement, and registrar for approval.

Once approved, the Registrar staff will tag you to your new program. Then, proceed to non-block sectioning enrollment, and afterwards, surrender your old RFID to OSFA office, and pay 250 pesos for reprinting and proceed to ID room and bring your blue slip from OSFA and OR for your new RFID.


Sources:
28-Additional Enrollment Steps (Shiftee).txt
29-Additional Enrollmen